In [1]:
"""
Task 14 - LangGraph Day 2
ReAct Agent using LangGraph

Reason -> Act -> Final Answer
"""

from typing import TypedDict

from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool


# -------------------------------------------------
# Define Agent State
# -------------------------------------------------
class AgentState(TypedDict):
    question: str
    thought: str
    action: str
    observation: str
    answer: str


# -------------------------------------------------
# Create LLM
# -------------------------------------------------
llm = ChatOllama(
    model="llama3",
    temperature=0
)


# -------------------------------------------------
# Define Tools
# -------------------------------------------------
@tool
def calculator(expression: str) -> str:
    """Evaluate a math expression."""
    try:
        return str(eval(expression))
    except Exception:
        return "Invalid expression"


TOOLS = {
    "calculator": calculator
}


# -------------------------------------------------
# Reasoning Prompt
# -------------------------------------------------
reason_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a ReAct agent.\n"
            "Decide what action to take.\n"
            "Available actions:\n"
            "- calculator\n\n"
            "Respond ONLY in this format:\n"
            "Action: <action_name>\n"
            "Input: <action_input>"
        ),
        ("human", "{question}")
    ]
)


# -------------------------------------------------
# Reason Node
# -------------------------------------------------
def reason(state: AgentState) -> AgentState:
    chain = reason_prompt | llm
    response = chain.invoke({"question": state["question"]}).content

    lines = response.splitlines()
    action = lines[0].replace("Action:", "").strip()
    action_input = lines[1].replace("Input:", "").strip()

    return {
        **state,
        "thought": response,
        "action": action,
        "observation": action_input
    }


# -------------------------------------------------
# Action Node
# -------------------------------------------------
def act(state: AgentState) -> AgentState:
    tool_name = state["action"]
    tool_input = state["observation"]

    if tool_name in TOOLS:
        result = TOOLS[tool_name].invoke({"expression": tool_input})
    else:
        result = "Unknown action"

    return {
        **state,
        "observation": result
    }


# -------------------------------------------------
# Final Answer Node
# -------------------------------------------------
def finish(state: AgentState) -> AgentState:
    return {
        **state,
        "answer": f"The result is {state['observation']}"
    }


# -------------------------------------------------
# Build LangGraph
# -------------------------------------------------
graph = StateGraph(AgentState)

graph.add_node("reason", reason)
graph.add_node("act", act)
graph.add_node("finish", finish)

graph.set_entry_point("reason")
graph.add_edge("reason", "act")
graph.add_edge("act", "finish")
graph.add_edge("finish", END)

app = graph.compile()


# -------------------------------------------------
# Run Agent
# -------------------------------------------------
if __name__ == "__main__":
    result = app.invoke(
        {
            "question": "What is 25 * 4 + 10?"
        }
    )

    print("\n========== FINAL ANSWER ==========\n")
    print(result["answer"])


========== FINAL ANSWER ==========

The result is 110
